In [8]:
# x.py
from helper import *
from nse_func import *

# Do assignments
a = assign_var('snp')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()

ib =  get_connected('snp', 'live')

with open(logpath+'ztest.log', 'w'):
    pass # clear the run log

util.logToFile(logpath+'ztest.log')

In [ ]:
# do_hist.py
def do_hist(ib, undId):
    '''Historize ohlc
    Args:
        (ib) as connection object
        (undId) as contractId for underlying symbol in int
    Returns:
        df_hist as dataframe
        pickles the dataframe by symbol name
    '''
    qc = ib.qualifyContracts(Contract(conId=int(undId)))[0]
    hist = ib.reqHistoricalData(contract=qc, endDateTime='', 
                                        durationStr='365 D', barSizeSetting='1 day',  
                                                    whatToShow='Trades', useRTH=True)
    df_hist = util.df(hist)
    df_hist = df_hist.assign(date=pd.to_datetime(df_hist.date, format='%Y-%m-%d'))
    df_hist.insert(loc=0, column='symbol', value=qc.symbol)
    df_hist = df_hist.sort_values('date', ascending = False).reset_index(drop=True)
    df_hist.to_pickle(fspath+'_'+qc.symbol+'_ohlc.pkl')
    return None

In [ ]:
# Download cboe weeklies to a dataframe
dls = "http://www.cboe.com/publish/weelkysmf/weeklysmf.xls"

snp100 = list(pd.read_html('https://en.wikipedia.org/wiki/S%26P_100', 
                           header=0, match='Symbol')[0].loc[:, 'Symbol'])
snp100 = [s.replace('.', ' ') if '.' in s else s  for s in snp100] # without dot in symbol

# read from row no 11, dropna and reset index
df_cboe = pd.read_excel(dls, header=12, 
                        usecols=[0,2,3]).loc[11:, :]\
                        .dropna(axis=0)\
                        .reset_index(drop=True)

# remove column names white-spaces and remap to IBKR
df_cboe.columns = df_cboe.columns.str.replace(' ', '')

# remove '/' for IBKR
df_cboe.Ticker = df_cboe.Ticker.str.replace('/', ' ', regex=False)

# make symbols
symbols = {s for s in df_cboe.Ticker if s not in blacklist if s in snp100}

stocks = [Stock(symbol=s, exchange=exchange, currency='USD') for s in symbols]

stkblks = [stocks[i: i+blk] for i in range(0, len(stocks), blk)] # blocks of stocks

# qualify the contracts
contracts = [ib.qualifyContracts(*s) for s in stkblks]
contracts = [contract for subl in contracts for contract in subl]

# get the undPrices
tickers = ib.reqTickers(*contracts)
undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}

# make the dataframe of symbols
df_cs = util.df(contracts).loc[:, ['conId', 'symbol']]
df_cs = df_cs.rename({'conId': 'undId'}, axis='columns')

df_csul = df_cs.assign(undPrice=[round(undPrices[s],2) for s in df_cs.symbol], lot=100)

# get the chains
qundc = [v for k, v in qcs_dict.items()]

rawchains = {}
with tqdm(total=len(qundc), file=sys.stdout, unit=' symbol') as tqc:
    for q in qundc:
        tqc.set_description(f"SNP option chains frm IBKR for {q.symbol.ljust(9)}")
        rawchains.update({
            q: ib.reqSecDefOptParams(underlyingSymbol=q.symbol, 
                                      underlyingSecType='STK', 
                                      underlyingConId=q.conId, 
                                      futFopExchange='')})
        tqc.update(1)

chains = {k.symbol: j for k, v in rawchains.items() for j in v if j.exchange == exchange}

# integrate the strikes and expiries
exps = {k: v.expirations for k, v in chains.items()}
strikes = {k: v.strikes for k, v in chains.items()}

es = [list(product([k1], v1, v2)) for k1, v1 in exps.items() for k2, v2 in strikes.items() if k1 == k2]

df_es = pd.DataFrame([s for r in es for s in r], columns=['symbol', 'expiry', 'strike'])

# make the dataframe to be in between min and max dte
df_es = df_es.assign(dte=[get_dte(d) for d in df_es.expiry])
df_es = df_es[df_es.dte.between(mindte, maxdte)].reset_index(drop=True)

# make the chains dataframe
df_chains = pd.merge(df_csul, df_es)

# put the rights
df_chains = pd.concat([df_chains.assign(right='P'), df_chains.assign(right='C')]).reset_index(drop=True)

# make the chains small
mask = (df_chains.right == 'P') & (df_chains.strike < df_chains.undPrice) | \
       (df_chains.right == 'C') & (df_chains.strike > df_chains.undPrice)
df_chains = df_chains[mask].reset_index(drop=True)

In [21]:
#... Historize
#_____________

qual_unds = list(qcs_dict.values()) # qualified underlyings

# historize individual underlyings
hists = [] # initialize a list of underlying histories. Will be empty.
with tqdm(total=len(qual_unds), file=sys.stdout, unit=' symbol') as tqh:
    for q in qual_unds:
        tqh.set_description(f"Getting OHLC hist frm IBKR for {q.symbol.ljust(9)}")
        hists.append(catch(lambda: do_hist(ib, q.conId)))  # makes ohlc.pkl
        tqh.update(1)

# Capture the ohlc pickles
ohlc_pkls = [f for f in listdir(fspath) if (f[-8:] == 'ohlc.pkl')]
df_ohlcs = pd.concat([pd.read_pickle(fspath+o) for o in ohlc_pkls]).reset_index(drop=True)

# put stdev in ohlc and consolidate
df_ohlcs = df_ohlcs.assign(stDev=df_ohlcs.groupby('symbol').close.transform(lambda x: x.expanding(1).std(ddof=0)))
df_ohlcs.to_pickle(fspath+'ohlcs.pkl')

Getting OHLC hist frm IBKR for CHTR     : 100%|███████████████████████████████████| 91/91 [03:04<00:00,  1.45s/ symbol]


In [119]:
# generate std dataframe
df = df_ohlcs[['symbol', 'stDev']]  # lookup dataframe
df = df.assign(dte=df.groupby('symbol').cumcount()) # get the cumulative count for location as dte
df.set_index(['symbol', 'dte'])

df1 = df_chains[['symbol', 'dte']]  # data to be looked at
df2 = df1.drop_duplicates()  # remove duplicates

df_std = df2.set_index(['symbol', 'dte']).join(df.set_index(['symbol', 'dte']))

# join to get std in chains
df_chainstd = df_chains.set_index(['symbol', 'dte']).join(df_std).reset_index()

In [128]:
# eliminate puts and calls out of threshold
put_mask = (df_chainstd.strike < (df_chainstd.undPrice-(df_chainstd.stDev*putstdmult)))
df_puts = df_chainstd[put_mask]

call_mask = (df_chainstd.strike > (df_chainstd.undPrice+(df_chainstd.stDev*callstdmult)))
df_calls = df_chainstd[call_mask]

In [136]:
df_opts = pd.concat([df_puts, df_calls]).reset_index(drop=True)

In [ ]:
# collect the option contracts
optcons = [Option(i.symbol, i.expiry, i.strike, i.right, 'SMART') for i in df_opts[['symbol', 'expiry', 'strike', 'right']].itertuples()]

# dice them into blocks
optblks = [optcons[i: i+blk] for i in range(0, len(optcons), blk)]

# qualify the contracts in blocks
cblks = []
with tqdm(total=len(optblks), file=sys.stdout, unit=' symbol') as tqo:
    for q in optblks:
        tqo.set_description(f"Qualifying chains {q[0].symbol.ljust(9)} to {q[len(q)-1].symbol.ljust(9)}")
        cblks.append([ib.qualifyContracts(s) for s in q])
        tqo.update(1)

opts = [z for x in cblks for z in x]

In [142]:
[z for x in cblks for z in x if z]

[[Option(conId=367618730, symbol='AAPL', lastTradeDateOrContractMonth='20190628', strike=130.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  190628P00130000', tradingClass='AAPL')],
 [Option(conId=367618737, symbol='AAPL', lastTradeDateOrContractMonth='20190628', strike=131.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  190628P00131000', tradingClass='AAPL')],
 [Option(conId=367618740, symbol='AAPL', lastTradeDateOrContractMonth='20190628', strike=132.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  190628P00132000', tradingClass='AAPL')],
 [Option(conId=367618750, symbol='AAPL', lastTradeDateOrContractMonth='20190628', strike=133.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  190628P00133000', tradingClass='AAPL')],
 [Option(conId=367618761, symbol='AAPL', lastTradeDateOrContractMonth='20190628', strike=134.0, right='P', multiplier='100',

In [9]:
df_mega.to_pickle(fspath+'base.pickle')
util.logging.info('_________________SNP BASE BUILD COMPLETE_________________________')

In [11]:
# get the max fall and rise
df_frs = df_mega.drop('strike', 1).drop_duplicates() # for fall rise std

mfrs = [((s, d) + get_maxfallrisestd(ib=ib, c=c, dte=d, tradingdays=tradingdays, durmult=durmult)) for s, c, d in zip(df_frs.symbol, df_frs.contract, df_frs.dte)]

df_mfrs = pd.DataFrame(mfrs, columns=['symbol', 'dte', 'lo52', 'hi52', 'Fall', 'Rise', 'Std'])

df_mega = df_mega.set_index(['symbol', 'dte']).join(df_mfrs.set_index(['symbol', 'dte'])).reset_index()
df_mega['loStd'] = df_mega.undPrice - df_mega['Std'].multiply(putstdmult)
df_mega['hiStd'] = df_mega.undPrice + df_mega['Std'].multiply(callstdmult)

# flter puts and calls by standard deviation
df_puts = df_mega[df_mega.strike < df_mega.loStd]
df_calls = df_mega[df_mega.strike > df_mega.hiStd]

df_puts = df_puts.assign(right='P')
df_calls = df_calls.assign(right='C')

# qualify the options
df = pd.concat([df_puts, df_calls]).reset_index(drop=True).drop('contract', axis=1)

# qualify the option contracts
optipl = [Option(s, e, k, r, exchange) for s, e, k, r in zip(df.symbol, df.expiration, df.strike, df.right)]
optblks = [optipl[i: i+blk] for i in range(0, len(optipl), blk)] # blocks of optipl
cblks = [ib.qualifyContracts(*s) for s in optblks]
contracts = [z for x in cblks for z in x]

# prepare the qualified options dataframe
dfq = util.df(contracts).iloc[:, 1:6]
dfq.columns=['optId', 'symbol', 'expiration', 'strike', 'right'] # rename columns
dfq = dfq.set_index(['symbol', 'expiration', 'strike', 'right']) # set index

# join the qualified options with main dataframe. Remove those without conId
df_opt = df.set_index(['symbol', 'expiration', 'strike', 'right']).join(dfq).dropna()

# convert optId to int for price and margin
df_opt = df_opt.assign(optId=df_opt.optId.astype('int32'))

# get the prices
opt_tickers = ib.reqTickers(*contracts)
optPrices = {t.contract.conId: t.marketPrice() for t in opt_tickers}

df_opt = df_opt.assign(optPrice=df_opt.optId.map(optPrices))
df_opt = df_opt[~(df_opt.optPrice <= 0)].reset_index() # remove options without a price

#... get the margins
# {conId: contract} dictionary
c_dict = {c.conId: c for c in contracts if c.conId in list(df_opt.optId)}
orders = [Order(action='SELL', orderType='MKT', totalQuantity=1, whatIf=True)]*len(c_dict)

# get the margins
margins = [float(catch(lambda: (ib.whatIfOrder(c, o).initMarginChange))) for c, o in zip(c_dict.values(), orders)]

df_opt = df_opt.assign(optMargin = margins)
df_opt = df_opt[df_opt.optMargin < 1e7] # remove options with very large margins

# get the rom
df_opt = df_opt.assign(rom=df_opt.optPrice*df_opt.lot/df_opt.optMargin*365/df_opt.dte).sort_values('rom', ascending=False)

NameError: name 'get_maxfallrisestd' is not defined